In [1]:
# !pwd

import plotly.io as pio
pio.renderers.default = "notebook"

In [1]:
import CRISP

# Simulation Utility

### Atomic Indices

In [3]:
from CRISP.simulation_utility.atomic_indices import run_atom_indices

help(run_atom_indices)


In [4]:
from CRISP.simulation_utility.atomic_indices import run_atom_indices

file_path = "./wrapped_traj.traj"
output_folder = './indices_new/'

cutoffs = {
    ("O", "H"): 1.2,
    ("Si", "O"): 1.8,
    ("Al", "Si"): 3.2,
    ("O", "O"): 3.0
}

run_atom_indices(file_path, output_folder, frame_index=10, cutoffs=cutoffs)

### Atomic_traj_linemap

In [5]:
from CRISP.simulation_utility.atomic_traj_linemap import plot_atomic_trajectory
help(plot_atomic_trajectory)

In [6]:
from CRISP.simulation_utility.atomic_traj_linemap import plot_atomic_trajectory

traj_file = "./wrapped_traj.traj"
output_dir = "./atomic_traj_linemap/o_atom_trajectory.html"
selected_atoms = [593]
frame_skip = 1  

fig = plot_atomic_trajectory(
    traj_path=traj_file, 
    selected_indices=selected_atoms,
    output_path=output_dir,
    frame_skip=frame_skip,
    plot_title="Oxygen Atom Movement Analysis",
    atom_size_scale=1.2,
    show_plot=False
)


### Error Analysis

In [7]:
from CRISP.simulation_utility.error_analysis import autocorrelation_analysis, block_analysis

help(autocorrelation_analysis)

In [8]:
help(block_analysis)

In [9]:
import numpy as np

data_positions = np.load("./error/positions.npy")
res_positions = autocorrelation_analysis(data_positions,plot_acf=True,max_lag=500)
print(res_positions)

In [10]:
import numpy as np

data_energy = np.loadtxt("./error/md_20k.log", skiprows=1, usecols=2)
acf_error = autocorrelation_analysis(data_energy, plot_acf=True)
block_error = block_analysis(data_energy, convergence_tol=0.001, plot_blocks=False)

print(acf_error)
print(block_error)


### Interatomic Distances

In [11]:
from CRISP.simulation_utility.interatomic_distances import distance_calculation

help(distance_calculation)

In [12]:
from CRISP.simulation_utility.interatomic_distances import save_distance_matrices

help(save_distance_matrices)

In [13]:
from CRISP.simulation_utility.interatomic_distances import distance_calculation

traj_path = "./wrapped_traj.traj"
frame_skip = 10
index_type = ["O"]  

full_dms, sub_dms = distance_calculation(traj_path, frame_skip, index_type)

In [14]:
save_distance_matrices(full_dms, sub_dms, index_type, output_dir="distance_calculations_zeo")

### Subsampling

In [15]:
from CRISP.simulation_utility.subsampling import subsample
help(subsample)

In [16]:
from CRISP.simulation_utility.subsampling import subsample

all_frames = subsample(
    filename="./Subsmapling/local_minima.traj",
    n_samples=30,
    index_type="all",
    file_format="traj",
    skip=10,
    plot_subsample=True,
    output_dir="./Subsmapling"
)


# Data Analysis

### CN_Calculation

In [2]:
from CRISP.data_analysis.contact_coordination import coordination
help(coordination)

In [3]:
filename = "./wrapped_traj.traj"
target_atoms = "O"
bonded_atoms = ["O"]
custom_cutoffs = {('Al', 'O'): 3.5, ('O', 'O'): 2.5}
skip_frames = 10

In [4]:
cn  = coordination(filename, 
             target_atoms, bonded_atoms, custom_cutoffs, 
             skip_frames=10, plot_cn=True, output_dir="./CN_data")

### Contacts

In [5]:
from CRISP.data_analysis.contact_coordination import contacts
help(contacts)

In [6]:
filename = "./wrapped_traj.traj"
target_atoms = "O"
bonded_atoms = ["O"]
custom_cutoffs = {('Al', 'O'): 3.5, ('O', 'O'): 2.5}
skip_frames = 1

In [7]:
sub_dm, cal_contacts = contacts(
    filename, target_atoms, bonded_atoms, custom_cutoffs,
    skip_frames=1,
    plot_distance_matrix=True,
    plot_contacts=True,
    time_step=50.0*1000,  # fs
    output_dir="./Contacts_data")

### Hydrogen_Bond_Calculation

In [8]:
from CRISP.data_analysis.h_bond import hydrogen_bonds
help(hydrogen_bonds)

In [9]:
from CRISP.data_analysis.h_bond import hydrogen_bonds

h_bonds_both_plots = hydrogen_bonds(
    filename="./wrapped_traj.traj",
    skip_frames=1,
    acceptor_atoms=["O"],
    angle_cutoff=120,
    mic=True,
    output_dir = "./H_Bond_Data",
    time_step=50*1000,
    plot_count=True,
    plot_heatmap=True,
    plot_graph_frame=True,        # Generate frame-specific plot
    plot_graph_average=True,      # Generate average plot
    graph_frame_index=10          # Use frame 10 instead of default 0
)

### Radial_Distribution_Function

In [10]:
from CRISP.data_analysis.prdf import analyze_rdf
help(analyze_rdf)

In [11]:
traj_file = "./wrapped_traj.traj"
rmax = 10.0
nbins = 50
frame_skip = 1
output_dir = "custom_ase"
output_filename = None  # or specify a filename like "output.pkl"
use_prdf = False  # Set to True if you want to calculate partial RDF
atomic_indices = None  # Set to a tuple of lists if use_prdf is True, e.g., ([10, 1])

data_rdf = analyze_rdf(
    use_prdf=use_prdf,
    rmax=rmax,
    traj_path=traj_file,
    nbins=nbins,
    frame_skip=frame_skip,
    output_filename=output_filename,
    atomic_indices=atomic_indices,
    output_dir=output_dir,
    plot_prdf=True
)

In [12]:
import pickle
import numpy as np

output_path = "./custom_ase/rdf_total.pkl"

with open(output_path, "rb") as f:
    data = pickle.load(f)

print("Data keys:", list(data.keys()))

x_data = data['x_data']
y_data_all = data['y_data_all']

print("x_data shape (bin centers):", x_data.shape)
print("Number of bins:", len(x_data))

print("y_data_all type:", type(y_data_all))
print("Number of frames:", len(y_data_all))


### Mean Square Displacement

In [20]:
from CRISP.data_analysis.msd import calculate_save_msd
help(calculate_save_msd)

In [21]:
from CRISP.data_analysis.msd import analyze_from_csv
help(analyze_from_csv)

In [22]:
from CRISP.data_analysis.msd import calculate_save_msd

traj_file = "../../../../BATH/System_Analysis_01ns/system_analysis/SiAl15/nvt.traj"
indices_file = "../../../../BATH/System_Analysis_01ns/system_analysis/SiAl15/indices_needed/ex_fram_ox.npy"
timestep = 50.0 * 100

msd_values, msd_times = calculate_save_msd(
    traj_file=traj_file,
    timestep_value=timestep,
    indices_file=indices_file,
    output_file="msd_results.csv",
    frame_skip=100
)

In [23]:
from CRISP.data_analysis.msd import analyze_from_csv
import pandas as pd

df = pd.read_csv("msd_results.csv")
print(f"Total data points in file: {len(df)}")

# Then use a valid fit_end value
D, error = analyze_from_csv(
    csv_file="msd_results.csv",
    fit_start=0,
    fit_end=len(df),  
    with_intercept=True,
    plot_msd=True
)

### Clustering

In [24]:
from CRISP.data_analysis.clustering import StructureAnalyzer
help(StructureAnalyzer)

In [18]:
from CRISP.data_analysis.clustering import analyze_trajectory
help(analyze_trajectory)

#### Frame 

In [17]:
from CRISP.data_analysis.clustering import StructureAnalyzer
import numpy as np
import os

traj_file = "../../../../BATH/System_Analysis_01ns/system_analysis/SiAl15/nvt.traj"
indices_file = "../../../../BATH/System_Analysis_01ns/system_analysis/SiAl15/indices_needed/ex_fram_ox.npy"
threshold = 3.0
min_samples = 3
output_dir = "SiAl15_clustering"

os.makedirs(output_dir, exist_ok=True)

atom_indices = np.load(indices_file)

analyzer = StructureAnalyzer(
    traj_file=traj_file,
    atom_indices=atom_indices,
    threshold=threshold,
    min_samples=min_samples,
    metric='precomputed',
    custom_frame_index=-1  
)

results = analyzer.analyze_structure(output_dir=output_dir)


#### Trajectory

In [16]:
from CRISP.data_analysis.clustering import analyze_trajectory, save_analysis_results, plot_analysis_results
import os
import numpy as np

traj_file = "../../../../BATH/System_Analysis_01ns/system_analysis/SiAl15/nvt.traj"
indices_file = "../../../../BATH/System_Analysis_01ns/system_analysis/SiAl15/indices_needed/ex_fram_ox.npy"
threshold = 3.0
min_samples = 3
skip_frames = 1000  
output_dir = "SiAl15_traj_analysis"
output_prefix = "SiAl15_traj_clusters"

os.makedirs(output_dir, exist_ok=True)

analysis_results = analyze_trajectory(
    trajectory_path=traj_file,
    atom_indices_path=indices_file,
    threshold=threshold,
    min_samples=min_samples,
    skip_frames=skip_frames,
    output_dir=output_dir,
    save_html_visualizations=True  # Save HTML visualizations of first and last frames
)

pickle_file = save_analysis_results(
    analysis_results=analysis_results,
    output_dir=output_dir,
    output_prefix=output_prefix
)

plot_analysis_results(pickle_file, output_dir=output_dir)